In [2]:
ruta='/Users/Leon/Documents/Data GH/Mineria de textos/corpusTASS-2020'

In [25]:
import pandas as pd
import os

# Cargar archivos
df_train = pd.read_csv(os.path.join(ruta, 'train.tsv'), sep='\t')
df_test = pd.read_csv(os.path.join(ruta, 'test.tsv'), sep='\t')
df_dev = pd.read_csv(os.path.join(ruta, 'dev.tsv'), sep='\t')



In [26]:

# Filtrar columnas y eliminar filas con valores nulos
df_train_filtered = df_train[['texto', 'etiqueta']].dropna()
df_dev_filtered = df_dev[['texto', 'etiqueta']].dropna()
df_test_filtered = df_test[['texto', 'etiqueta']].dropna()

In [27]:
import re

# Haremos la limpieza de los textos con expresiones regulares

def limpiar_texto(texto):
    texto = re.sub(r'@\w+', '', texto)  # eliminar menciones
    texto = re.sub(r'http\S+|www\S+', '', texto)  # eliminar URLs
    texto = re.sub(r'#\w+', '', texto)  # eliminar hashtags
    #texto = re.sub(r'[^\w\sáéíóúüÁÉÍÓÚÜ]', '', texto)  # eliminar acentos y caracteres especiales
    texto = texto.lower()  # convertir a minúsculas
    texto = re.sub(r'\s+', ' ', texto).strip()  # eliminar espacios extra
    texto = texto.encode('ascii', 'ignore').decode('ascii')  # eliminar emojis y simbolos no ASCII
    return texto

In [28]:
df_train_filtered['texto'] = df_train_filtered['texto'].astype(str).apply(limpiar_texto)
df_dev_filtered['texto'] = df_dev_filtered['texto'].astype(str).apply(limpiar_texto)
df_test_filtered['texto'] = df_test_filtered['texto'].astype(str).apply(limpiar_texto)


In [29]:
from datasets import Dataset
#convertimos a formato dataset
dataset_train = Dataset.from_pandas(df_train_filtered.reset_index(drop=True))
dataset_dev = Dataset.from_pandas(df_dev_filtered.reset_index(drop=True))
dataset_test = Dataset.from_pandas(df_test_filtered.reset_index(drop=True))

In [11]:
from transformers import pipeline


# Usa un modelo en español compatible con análisis de sentimiento desde Hugging Face
classifier = pipeline("text-classification", model="finiteautomata/beto-sentiment-analysis")

# Aplica el modelo a los textos del corpus para predecir el sentimiento.
def clasificar_sentimiento(ejemplo):
    resultado = classifier(ejemplo['texto'])
    return {'prediccion_modelo': resultado[0]['label'].lower()}  # minúscula para facilitar comparación

# Probamos
dataset_train = dataset_train.map(clasificar_sentimiento)
#dataset_dev = dataset_dev.map(clasificar_sentimiento)
#dataset_test = dataset_test.map(clasificar_sentimiento)

# Mostramos
print(dataset_train[0])



Map: 100%|██████████| 4802/4802 [02:52<00:00, 27.86 examples/s]

{'texto': 'jajajaja eso es verdad aqu no hay uno cuerdo', 'etiqueta': 'N', 'prediccion_modelo': 'neg'}


In [15]:
dataset_train[3]

{'texto': 'mi ltima partida jugada con sona support la grandes razones para jugar sona',
 'etiqueta': 'P',
 'prediccion_modelo': 'pos'}

In [16]:
#Cuadramos etiquetas y predicciones
mapa_etiquetas = {'P': 'pos', 'NEU': 'neu', 'N': 'neg'}
y_true = [mapa_etiquetas[e] for e in dataset_train['etiqueta']]
y_pred = dataset_train['prediccion_modelo']

In [19]:
for i in range(0,40): print(y_true[i],y_pred[i]) 

neg neg
neu neu
neu neu
pos pos
pos neu
pos pos
neg neg
neu neu
pos pos
pos pos
neg neg
neu neu
neg neg
neu neu
pos pos
pos pos
neu neu
neg neg
neu neu
neu neu
neg neg
pos pos
neu neu
neg neu
neu neu
neg neg
neu neu
neg neg
pos pos
pos neu
neg neg
pos pos
pos pos
pos pos
neg neg
pos pos
neu neu
neg neg
neg neg
neg neg


In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix




print("Exactitud:", accuracy_score(y_true, y_pred))
print("\n Reporte de clasificación:")
print(classification_report(y_true, y_pred, digits=3))

#  Matriz de confusión
etiquetas = ['pos', 'neu', 'neg']
cm = confusion_matrix(y_true, y_pred, labels=etiquetas)


Exactitud: 0.9379425239483549

 Reporte de clasificación:
              precision    recall  f1-score   support

         neg      0.955     0.949     0.952      1885
         neu      0.898     0.938     0.917      1523
         pos      0.962     0.923     0.942      1394

    accuracy                          0.938      4802
   macro avg      0.938     0.937     0.937      4802
weighted avg      0.939     0.938     0.938      4802



In [23]:
import random

# Crear listas comparables
etiquetas_reales = [mapa_etiquetas[e] for e in dataset_train['etiqueta']]
predicciones = dataset_train['prediccion_modelo']
textos = dataset_train['texto']

# Buscar aciertos y errores
aciertos = []
errores = []

for texto, real, pred in zip(textos, etiquetas_reales, predicciones):
    if real == pred:
        aciertos.append((texto, real, pred))
    else:
        errores.append((texto, real, pred))

# Mostrar ejemplos aleatorios
print("Ejemplos correctos:")
for texto, real, pred in random.sample(aciertos, min(3, len(aciertos))):
    print(f"\nTexto: {texto}\nEtiqueta real: {real} | Predicción: {pred}")

print("\n Ejemplos equivocados:")
for texto, real, pred in random.sample(errores, min(3, len(errores))):
    print(f"\nTexto: {texto}\nEtiqueta real: {real} | Predicción: {pred}")


Ejemplos correctos:

Texto: buen da pap te amo mucho
Etiqueta real: pos | Predicción: pos

Texto: me hacen quedar como un zorro lpm pero todas las minas que me nombran estn re buenas matense
Etiqueta real: neg | Predicción: neg

Texto: nos fue bien en el parcial solo nos faltan 4
Etiqueta real: pos | Predicción: pos

 Ejemplos equivocados:

Texto: a todas las personas que fueron el mejor respaldo este 2016 sobre todo a ti amor
Etiqueta real: neu | Predicción: pos

Texto: la verdad extrao mucho mi vida universitaria
Etiqueta real: neg | Predicción: neu

Texto: feliz cumpleaos julian z q dios te bendiga muchsimo un fuerte abrazo desde huacho
Etiqueta real: neu | Predicción: pos


In [ ]:
 # Usa un modelo en español compatible con análisis de sentimiento desde Hugging Face
classifier = pipeline("text-classification", model="finiteautomata/beto-sentiment-analysis")

# Aplica el modelo a los textos del corpus para predecir el sentimiento.
def clasificar_sentimiento(ejemplo):
    resultado = classifier(ejemplo['texto'])
    return {'prediccion_modelo': resultado[0]['label'].lower()}  # minúscula para facilitar comparación

# Probamos
dataset_train = dataset_train.map(clasificar_sentimiento)
#dataset_dev = dataset_dev.map(clasificar_sentimiento)
#dataset_test = dataset_test.map(clasificar_sentimiento)

# Mostramos
print(dataset_train[0])

/Users/Leon/Library/Python/3.9/lib/python/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Map:  24%|██▍       | 1143/4802 [00:48<02:17, 26.61 examples/s]

In [ ]:



print("Exactitud:", accuracy_score(y_true, y_pred))
print("\n Reporte de clasificación:")
print(classification_report(y_true, y_pred, digits=3))

#  Matriz de confusión
etiquetas = ['pos', 'neu', 'neg']
cm = confusion_matrix(y_true, y_pred, labels=etiquetas)


In [ ]:

# Crear listas comparables
etiquetas_reales = [mapa_etiquetas[e] for e in dataset_train['etiqueta']]
predicciones = dataset_train['prediccion_modelo']
textos = dataset_train['texto']

# Buscar aciertos y errores
aciertos = []
errores = []

for texto, real, pred in zip(textos, etiquetas_reales, predicciones):
    if real == pred:
        aciertos.append((texto, real, pred))
    else:
        errores.append((texto, real, pred))

# Mostrar ejemplos aleatorios
print("Ejemplos correctos:")
for texto, real, pred in random.sample(aciertos, min(3, len(aciertos))):
    print(f"\nTexto: {texto}\nEtiqueta real: {real} | Predicción: {pred}")

print("\n Ejemplos equivocados:")
for texto, real, pred in random.sample(errores, min(3, len(errores))):
    print(f"\nTexto: {texto}\nEtiqueta real: {real} | Predicción: {pred}")